In [17]:
import pandas as pd
import os

In [18]:
한국재무정보 = pd.read_excel('데이터클렌징_재무정보_한국_V01_20220106.xlsx')

In [19]:
'0'*0 + '안녕하세요'

'안녕하세요'

In [20]:
한국재무정보['상장코드'] = 한국재무정보['상장코드'].apply(lambda x : (6-len(str(x)))*'0'+str(x))

# Part1 Crawling Module 

referemce : 

https://jenn1won.tistory.com/20

In [21]:
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException 
import time 
import re 
from tqdm import tqdm 
import selenium
import bs4
from selenium import webdriver
from bs4 import BeautifulSoup
import html5lib
import requests
import math

In [22]:
# 하나의 페이지에서 뉴스 20개 긁어오기 
def get_naver_news_urls(fund, search, source): 
    # source(html)를 입력으로 
    results = [] 
    soup = BeautifulSoup(source, 'html.parser') 
    title_list = soup.find_all(attrs={'class':'articleSubject'}) 
    press_list = soup.find_all(attrs={'class' : 'press'})
    date_list = soup.find_all(attrs={'class' : 'wdate'})
    
    for index,li in enumerate(title_list): 
        title = 'NA' 
        naver_url = 'NA' 
        fund = fund 
        search = search 
        title = li.text.strip() 
        url = 'https://finance.naver.com'+li.find('a').get('href') 
        press = press_list[index].text
        date = date_list[index].text.strip()[0:10]
        date = date.replace('-','')
        results.append([fund, search, title, url,press,date]) 
    
    return results

In [23]:
def get_1(fund, search): 
    page1_results = [] 
    total_results = [] 
    driver.get('https://finance.naver.com/news/')  
    search = search
    driver.find_element_by_xpath('//*[@id="newsMainTop"]/div/div[2]/form/div/input').click()           #검색창클릭
    driver.find_element_by_xpath('//*[@id="newsMainTop"]/div/div[2]/form/div/input').send_keys(search) #종목입력 
    driver.find_element_by_xpath('//*[@id="newsMainTop"]/div/div[2]/form/div/a/input').click()         #검색클릭
    time.sleep(0.5) 
    
    myurl = driver.current_url 
    try : 
        for page in range(1,5) : 
            # 100개 #print(str(page)+'크롤링 시작') 
            source=driver.page_source 
            page1_results = get_naver_news_urls(fund,search,source) 
            # print(results, len(results)) 
            total_results.extend(page1_results) 
            
            # 다음 페이지 
            url = myurl+'&page='+str(page+1) 
            driver.get(url) 
            time.sleep(0.5) 
            
    except NoSuchElementException: 
        next                 
    
    return total_results

# Part 2 Crawling

In [24]:
path = "C:\\Users\\user\\chromedriver.exe"
driver = webdriver.Chrome(path)
# 크롬 웹 드라이버 객체 생성.
# 크롬드라이버로 크롬을 조작.

driver.get("https://finance.naver.com")
#드라이버 객체에서 원하는 url로 이동.

In [25]:
company_news_all = pd.DataFrame(columns = ['현지언어기업명','상장코드','제목','이벤트URL','정보제공자','이벤트 행사 날짜'])

for iter_num, company_code in enumerate(tqdm(한국재무정보['상장코드'])) : 
    fund = 한국재무정보['현지언어기업명'][iter_num]
    company_news = pd.DataFrame(get_1(fund, company_code), columns = ['현지언어기업명','상장코드','제목','이벤트URL','정보제공자','이벤트 행사 날짜'])
    company_news_all = pd.concat([company_news_all, company_news])
    
    
    if (iter_num+1)%100 == 0 :
        file_name = '기업뉴스'+str(iter_num+1)+'.xlsx'
        company_news_all['이벤트 기록 날짜'] = company_news_all['이벤트 행사 날짜']
        company_news_all['이벤트종류'] = '뉴스/공시'
        company_news_all.to_excel(file_name)
        #리셋
        company_news_all = pd.DataFrame(columns = ['현지언어기업명','상장코드','제목','이벤트URL','정보제공자','이벤트 행사 날짜'])
        
    if (iter_num+1) == 2267 : 
        file_name = '기업뉴스'+str(iter_num+1)+'.xlsx'
        company_news_all['이벤트 기록 날짜'] = company_news_all['이벤트 행사 날짜']
        company_news_all['이벤트종류'] = '뉴스/공시'
        company_news_all.to_excel(file_name)
        #리셋
        

100%|████████████████████████████████████████████████████████████████████████████| 2267/2267 [3:13:37<00:00,  5.12s/it]
